# **Rede Neural**

In [ ]:
import numpy as np

class RedeNeural:
    def __init__(self, estrutura):
        self.estrutura = estrutura
        self.pesos = []
        self.vies = []
        for i in range(len(estrutura) - 1):
            # Inicialização Xavier
            limite = np.sqrt(6 / (estrutura[i] + estrutura[i + 1]))
            self.pesos.append(np.random.uniform(-limite, limite, (estrutura[i], estrutura[i + 1])))
            self.vies.append(np.zeros((1, estrutura[i + 1])))



    def relu(self, z):

        return np.maximum(0, z)

    def derivada_relu(self, z):

        return (z > 0).astype(float)

    def erro_quadratico_medio(self, y_pred, y_true):

        return np.mean((y_pred - y_true) ** 2)

    def derivada_eqm(self, y_pred, y_true):
        return 2 * (y_pred - y_true) / y_true.size

    def feedforward(self, entrada):
        ativacoes = [entrada]
        z_camadas = []
    
        for i in range(len(self.pesos)):
            z = np.dot(ativacoes[-1], self.pesos[i]) + self.vies[i]
            z_camadas.append(z)
            if i == len(self.pesos) - 1:
                ativacao = z
            else:
                ativacao = self.relu(z)
            ativacoes.append(ativacao)
    
        return ativacoes, z_camadas


    def backpropagation(self, entrada, saida_real, taxa_aprendizado):
    
        # Passo 1: Feedforward
        ativacoes, z_camadas = self.feedforward(entrada)
    
        # Passo 2: Calcula erro da saída
        erro_saida = self.derivada_eqm(ativacoes[-1], saida_real)
    
        deltas = [erro_saida]
    
        # Passo 3: Retropropagação do erro
        for i in range(len(self.estrutura) - 2, 0, -1):
            erro = np.dot(deltas[0], self.pesos[i].T) * self.derivada_relu(z_camadas[i - 1])
            deltas.insert(0, erro)
    
        # Passo 4: Atualização de pesos e vieses
        for i in range(len(self.pesos)):
            self.pesos[i] -= taxa_aprendizado * np.dot(ativacoes[i].T, deltas[i])
            self.vies[i] -= taxa_aprendizado * np.sum(deltas[i], axis=0, keepdims=True)


: 

---
# **Regressão**
---

### Carregamento do Dataset - California Housing Prices

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import numpy as np

def carregar_dados_regressao():
    df = pd.read_csv('housing.csv')
    
    # TODO arrumar depois - ignorando ocean_proximity
    df = df.drop(columns=['ocean_proximity'])
    
    df = df.fillna(df.mean())
    
    X = df.drop(columns=['median_house_value']).values
    y = df['median_house_value'].values.reshape(-1, 1)
    

    # normaliza os dados
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()
    X = scaler_X.fit_transform(X)
    y = scaler_y.fit_transform(y)

    X_treino, X_teste, y_treino, y_teste = train_test_split(
        X, y, test_size=0.2, random_state=42)

    return X_treino, X_teste, y_treino, y_teste, scaler_y

# configura os dados
X_treino, X_teste, y_treino, y_teste, scaler_y = carregar_dados_regressao()


### Treinamento

In [ ]:
estrutura_camadas = [X_treino.shape[1], 50, 1]  # Camada oculta com 50 neurônios
rede = RedeNeural(estrutura_camadas)

# Treinamento a rede
def treinar_rede_regressao_ajustada(rede, X_treino, y_treino, X_teste, y_teste, epochs=2000, taxa_aprendizado=0.005):
    erros_treino = []
    erros_teste = []

    for epoca in range(epochs):
        # Treinar com o conjunto de treino
        rede.backpropagation(X_treino, y_treino, taxa_aprendizado)

        # Calcular erro no treino e no teste
        ativacoes_treino, _ = rede.feedforward(X_treino)
        erro_treino = rede.erro_quadratico_medio(ativacoes_treino[-1], y_treino)
        erros_treino.append(erro_treino)

        ativacoes_teste, _ = rede.feedforward(X_teste)
        erro_teste = rede.erro_quadratico_medio(ativacoes_teste[-1], y_teste)
        erros_teste.append(erro_teste)

        # Exibir progresso a cada 100 épocas
        if epoca % 100 == 0:
            print(f"Época {epoca}: Erro de Treino = {erro_treino:.4f}, Erro de Teste = {erro_teste:.4f}")

    return erros_treino, erros_teste


# Executar o treinamento
erros_treino, erros_teste = treinar_rede_regressao_ajustada(rede, X_treino, y_treino, X_teste, y_teste)


### Visualização dos Resultados

In [ ]:
import matplotlib.pyplot as plt

# Plotar erros de treino e teste
plt.figure(figsize=(10, 6))
plt.plot(erros_treino, label="Erro de Treino")
plt.plot(erros_teste, label="Erro de Teste")
plt.xlabel("Épocas")
plt.ylabel("Erro Quadrático Médio")
plt.title("Erro durante o Treinamento")
plt.legend()
plt.show()

---
# **Classificação Binária**
---

### Carregamento do Dataset - PIMA Indians Diabetes Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

df = pd.read_csv('diabetes.csv')

X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Treinamento

In [ ]:
class RedeNeuralClassificacao(RedeNeural):
    def __init__(self, estrutura):
        super().__init__(estrutura)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def derivada_sigmoid(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def feedforward(self, entrada):
        ativacoes = [entrada]
        z_camadas = []

        for peso, vies in zip(self.pesos, self.vies):
            z = np.dot(ativacoes[-1], peso) + vies
            z_camadas.append(z)
            ativacao = self.sigmoid(z)
            ativacoes.append(ativacao)

        return ativacoes, z_camadas

    def backpropagation(self, entrada, saida_real, taxa_aprendizado):
        ativacoes, z_camadas = self.feedforward(entrada)
        erro_saida = (ativacoes[-1] - saida_real) * self.derivada_sigmoid(z_camadas[-1])

        deltas = [erro_saida]

        for i in range(len(self.estrutura) - 2, 0, -1):
            erro = np.dot(deltas[0], self.pesos[i].T) * self.derivada_sigmoid(z_camadas[i-1])
            deltas.insert(0, erro)

        for i in range(len(self.pesos)):
            self.pesos[i] -= taxa_aprendizado * np.dot(ativacoes[i].T, deltas[i])
            self.vies[i] -= taxa_aprendizado * np.sum(deltas[i], axis=0, keepdims=True)
            

# configura
estrutura_camadas = [X_train.shape[1], 64, 1]  # camada oculta - 64 neurônios
rede_classificacao = RedeNeuralClassificacao(estrutura_camadas)


# treina
def treinar_rede_classificacao(rede, X_train, y_train, X_test, y_test, epochs=1000, taxa_aprendizado=0.01):
    for epoch in range(epochs):
        rede.backpropagation(X_train, y_train.values.reshape(-1,1), taxa_aprendizado)
        if epoch % 100 == 0:
            # calcula as previsões para treino e teste
            y_pred_train = rede.feedforward(X_train)[0][-1]
            y_pred_test = rede.feedforward(X_test)[0][-1]

            # calcula o erro
            erro_treino = rede.erro_quadratico_medio(y_pred_train, y_train.values.reshape(-1,1))
            erro_teste = rede.erro_quadratico_medio(y_pred_test, y_test.values.reshape(-1,1))

            print(f"Época {epoch}: Erro de treino = {erro_treino:.4f} - Erro de teste = {erro_teste:.4f}")

# Executa
treinar_rede_classificacao(rede_classificacao, X_train, y_train, X_test, y_test)

### Visualização dos Resultados

In [ ]:
# Plotar erros de treino e teste
plt.figure(figsize=(10, 6))
plt.plot(erros_treino, label="Erro de Treino")
plt.plot(erros_teste, label="Erro de Teste")
plt.xlabel("Épocas")
plt.ylabel("Erro Quadrático Médio")
plt.title("Erro durante o Treinamento")
plt.legend()
plt.show()

---
# **Classificação Multiclasse**
---

### Carregamento do Dataset - Iris

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('iris.csv')

X = df.drop(columns=["Id", "Species"])
y = df["Species"]

encoder = OneHotEncoder()
y = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Treinamento

In [ ]:
import numpy as np

class RedeNeuralMulticlasse:
    def __init__(self, estrutura):
        self.estrutura = estrutura
        self.pesos = []
        self.vies = []
        for i in range(len(estrutura) - 1):
            limite = np.sqrt(6 / (estrutura[i] + estrutura[i + 1]))
            self.pesos.append(np.random.uniform(-limite, limite, (estrutura[i], estrutura[i + 1])))
            self.vies.append(np.zeros((1, estrutura[i + 1])))

    def relu(self, z):
        return np.maximum(0, z)

    def derivada_relu(self, z):
        return (z > 0).astype(float)

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_pred, y_true):
        return -np.mean(np.sum(y_true * np.log(y_pred + 1e-15), axis=1))

    def feedforward(self, entrada):
        ativacoes = [entrada]
        z_camadas = []

        for i in range(len(self.pesos)):
            z = np.dot(ativacoes[-1], self.pesos[i]) + self.vies[i]
            z_camadas.append(z)
            if i == len(self.pesos) - 1:
                ativacao = self.softmax(z)
            else:
                ativacao = self.relu(z)
            ativacoes.append(ativacao)

        return ativacoes, z_camadas

    def backpropagation(self, entrada, saida_real, taxa_aprendizado):
        ativacoes, z_camadas = self.feedforward(entrada)

        erro_saida = ativacoes[-1] - saida_real
        deltas = [erro_saida]

        for i in range(len(self.pesos) - 1, 0, -1):
            delta = np.dot(deltas[0], self.pesos[i].T) * self.derivada_relu(z_camadas[i - 1])
            deltas.insert(0, delta)

        for i in range(len(self.pesos)):
            gradiente_peso = np.dot(ativacoes[i].T, deltas[i])
            gradiente_vies = np.sum(deltas[i], axis=0, keepdims=True)

            self.pesos[i] -= taxa_aprendizado * gradiente_peso
            self.vies[i] -= taxa_aprendizado * gradiente_vies

In [ ]:
# configura a rede
estrutura_camadas = [X_train.shape[1], 64, y_train.shape[1]]
rede_multiclasse = RedeNeuralMulticlasse(estrutura_camadas)

def treinar_rede_multiclasse(rede, X_train, y_train, X_test, y_test, epochs=2000, taxa_aprendizado=0.01):
    erros_treino = []
    erros_teste = []


    X_train = np.asarray(X_train)
    y_train = y_train.toarray() if hasattr(y_train, "toarray") else np.asarray(y_train)
    X_test = np.asarray(X_test)
    y_test = y_test.toarray() if hasattr(y_test, "toarray") else np.asarray(y_test)

    for epoch in range(epochs):
        rede.backpropagation(X_train, y_train, taxa_aprendizado)

        if epoch % 100 == 0 or epoch == epochs - 1:
            y_pred_train = rede.feedforward(X_train)[0][-1]
            y_pred_test = rede.feedforward(X_test)[0][-1]

            erro_treino = rede.cross_entropy_loss(y_pred_train, y_train)
            erro_teste = rede.cross_entropy_loss(y_pred_test, y_test)

            erros_treino.append(erro_treino)
            erros_teste.append(erro_teste)

            print(f"Época {epoch}: Erro de treino = {erro_treino:.4f}, Erro de teste = {erro_teste:.4f}")

    return erros_treino, erros_teste


# treinamento da rede
erros_treino, erros_teste = treinar_rede_multiclasse(rede_multiclasse, X_train, y_train, X_test, y_test)



### Visualização dos Resultados

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(erros_treino, label="Erro de Treino")
plt.plot(erros_teste, label="Erro de Teste")
plt.xlabel("Épocas")
plt.ylabel("Erro Quadrático Médio")
plt.title("Erro durante o Treinamento")
plt.legend()
plt.show()